you'll need to install anaconda and create an environment

use of "conda install" for all packages installations is recommended to avoid conflicts between packages dependencies

download anaconda
https://www.anaconda.com/download

managing environments
https://conda.io/activation

In [ ]:
#importing packages
from matplotlib import *
import numpy as np
import operator
from __future__ import division
%matplotlib inline
import pandas as pd
import csv
from matplotlib import pyplot as plt
from scipy.stats import linregress, spearmanr, chi2, beta
from collections import Counter, defaultdict
from scipy.stats.contingency import odds_ratio
import os, sys, re, calendar, datetime
import subprocess
from datetime import date, timedelta
import datetime
import os, sys
import subprocess

In [ ]:
#paths to influweb files, both intake and weekly_responses datasets

#v2 (namely "22-12-1") file path
path='/v2'
#v1 (namely "21-11-1") file path
path_prev='/v1'
#v0 file path
path_prev2='/v0'
    

In [ ]:
#functions definitions, we'll need them later
#some basic data cleaning, homogenize answers to True and False

## turn all f, FALSE to True and False
def translate(entry):
    if entry=='f':
        entry=False
    elif entry=='FALSE':
        entry=False
    elif entry=='t':
        entry=True
    elif entry=='TRUE':
        entry=True
    return entry

## Convert weeks with formats like 2014-2 to zero-padded format 2014-02
def fix_yearweek(date_string):
    year, week =  map(lambda x: int(x), date_string.split('-'))
    fixed = "{0}-{1:02d}".format(year, week)
    
    assert re.match("\d{4}-\d{2}$", fixed)
    return fixed
#######################

#######################
def weekMinus(week, minusweek):
    #week is a string 'yyyy-w'
    yr, wk = map(lambda x: int(x), week.split('-'))
    mid_date = getMiddleDayOfWeek(yr, wk)
    res_date = mid_date - datetime.timedelta(days=7*minusweek)
    isoyear, isoweek, isoday = res_date.isocalendar()
    return str(isoyear) + '-' + str(isoweek)
#######################

#######################
def getMiddleDayOfWeek(year, week):
    d = datetime.date(year,1,1)
    if(d.weekday()>3):
        d = d+datetime.timedelta(7-d.weekday())
    else:
        d = d - datetime.timedelta(d.weekday())
    dlt = datetime.timedelta(days = (week-1)*7)
    return d + dlt + datetime.timedelta(days=3)
#######################

#######################
def weekPlusOne(week):
    #week is a string 'yyyy-w'
    yr, wk = map(lambda x: int(x), week.split('-'))
    mid_date = getMiddleDayOfWeek(yr, wk)
    res_date = mid_date + datetime.timedelta(days=7)
    isoyear, isoweek, isoday = res_date.isocalendar()
    return str(isoyear) + '-' + str(isoweek)
#######################

#######################
def days_difference(date1, date2):
    y1,m1,d1 = map(lambda x: int(x), date1.split('-'))
    y2,m2,d2 = map(lambda x: int(x), date2.split('-'))
    delta_days = (datetime.date(y1,m1,d1) - datetime.date(y2,m2,d2)).days
    return delta_days
#######################

#######################
#define if participant is showing ILI based on declared symptoms
def get_ILI_ECDC(row):
    ILI=False
    if row.Q1_0==False:
        if row.Q5==0.0 or row.Q6b==0.0: #0 = sudden onset
            if row.Q1_1==True or row.Q1_2==True or row.Q1_11==True or row.Q1_8==True or row.Q1_9==True:
                if row.Q1_5==True or row.Q1_6==True or row.Q1_7==True:
                    ILI=True  
    return ILI

#######################

def get_ARI(row):
    ARI=False
    if row.Q1_0==False:
        if row.Q5==0.0: #0 = sudden onset
            if row.Q1_6==True or row.Q1_5==True or row.Q1_7==True or row.Q1_3==True:
                ARI=True  
    return ARI

#######################
def get_any_symp(row):
    symp=False
    if row.Q1_0==False:
        symp=True  
    return symp
#######################

def get_fever(row):
    FEV=False
    if row.Q1_0==False:
        if row.Q1_1==True:
            FEV=True  
    return FEV
#######################

def get_phlegm(row):
    phlegm=False
    if row.Q1_0==False:
        if row.Q1_7==True:
            phlegm=True  
    return phlegm


#######################
def get_onset_date( submission_date, symptoms_date, fever_date ):
    onset_date = submission_date
    ## if symptoms onset date is available
    if pd.notnull(symptoms_date): 
        if 0 <= days_difference(submission_date, symptoms_date) <= 15:
            onset_date = symptoms_date
    ## otherwise use fever date onset. if none of them, return nan
    elif pd.notnull(submission_date) and pd.notnull(fever_date):
         if 0 <= days_difference(submission_date, fever_date) <= 15:
            onset_date = fever_date    
    return onset_date
#######################

#######################
def get_week_of_activity(global_id, submission_weeks):
    activity_weeks = []
    for week in submission_weeks: 
        wk_start, wk_end = fix_yearweek(weekMinus(week, 2)), fix_yearweek(weekMinus(week, -2))
        wk = wk_start
        while(wk <= wk_end):
            activity_weeks.append(wk)
            wk = fix_yearweek(weekPlusOne(wk))
    return sorted(set(activity_weeks))
#######################


###
# Return previous week (es: lastweek(datetime.datetime.strptime('18112019', "%d%m%Y").date()) -> 2019-46)
###
def lastweek(today=date.today()):
    for i in range(7):
        x = str((today - timedelta(days=i)).isocalendar()[0])+'-'+str((today - timedelta(days=i)).isocalendar()[1])
        d1 = str(today.isocalendar()[0])+'-'+str(today.isocalendar()[1])
        if x!=d1:
              return fix_yearweek(x)


In [ ]:
##
# Get season from week consindering the start of a new season from week week_th (int) (es: get_season_fromweek('2015-40', 45) -> 2014-2015)
##
def get_season_fromweek(week, week_th):
    w_y_list = week.split('-')
    if int(w_y_list[1]) >=week_th:         #es. week_th=45 -> new season after 45 week
        season = str(w_y_list[0])+'-'+str(int(w_y_list[0])+1)
    else:
        season = str(int(w_y_list[0])-1)+'-'+str(w_y_list[0])
    return season

def week_from_date(data):
    x = datetime.datetime.strptime(data, '%Y-%m-%d')
    if len(str(x.isocalendar()[1]))==2:
        w = str(x.isocalendar()[0])+'-'+str(x.isocalendar()[1])
    else:
        w = str(x.isocalendar()[0])+ '-0' +str(x.isocalendar()[1])
    return w

def get_epoch(x):
    p = '%Y-%m-%d %H:%M:%S'
    mytime = str(x)[:19]
    epoch = datetime.datetime(1970, 1, 1)
    tempo = int((datetime.datetime.strptime(mytime, p) - epoch).total_seconds())
    return tempo

def get_age(x):
    subm=int(x.intake_submission[:4])
    if any([x.version=='22-12-2', x.version=='21-11-1']):
        new_year= int(datetime.datetime.fromtimestamp(int(x.Q2)).strftime('%Y'))
        if subm - new_year < 0:
            print(x.Q2)
        return subm - new_year
    
    else:
        if '/' in x.Q2 or '-' in x.Q2:
            year=int(x.Q2[:4])
            if subm - year < 0:
                print(x.Q2)
            return subm - year
        
## define age classes from IN data
def get_age_class(x):
    if x<18 and x>0:
        return '<18'
    elif x>=18 and x<=40:
        return '18-40'
    elif x>40 and x<=65:
        return '41-65'
    elif x>65:
        return '>65'
    else: return 'nan'
    
## extract real population pyramid from census data
def get_iss_age_classes(x):
    if x!='Y_GE100':
        xx=int(x.strip('Y'))
        if xx<=18:
            y='<18'
        elif xx>18 and xx<=40: #maybe 44 is the standard
            y='18-40'
        elif xx>40 and xx<=65:
            y='41-65'
        elif xx>65:
            y='>65'
        else:
            y='rest'
        return y
    else: return 'rest'

def occupation(x):
    if str(x)!='nan':
        x=int(x)
        if x==0:
            return 'full_time'
        elif x==1:
            return 'part_time'
        elif x==2:
            return 'self-employed'
        elif x==3:
            return 'student'
        elif x==4:
            return 'homemaker'
        elif x==5:
            return 'unemployed'
        elif x==6:
            return 'on leave'
        elif x==7:
            return 'retired'
        elif x==8:
            return 'other'
    
def schooling(x):
    if x==0:
        return 'none'
    elif x==1:
        return 'int_school'
    elif x==2:
        return 'high_school'
    elif x==3:
        return 'bachelor'
    elif x==4:
        return 'master_phd'
    elif x==5:
        return 'student'
    
def unite(x):
    #print(x)
    aa=np.where(x)[0]
    if len(aa)>0:
        return int(aa[0])
        print(aa)
    else: return np.nan

### Define season and previous season one by one

In [ ]:
dates = datetime.date.today()
last_week = lastweek(dates)
print(f"Last week is: {last_week}")

In [ ]:
country = 'Italy'
country_code = 'IT'
rescaling = 1000. #applies to incidence
YEAR_MIN, YEAR_MAX = 2011, 2024 #

dates = datetime.date.today()

last_week = lastweek(dates)
print(f"Computing incidence for week: {last_week}")

my_season = get_season_fromweek(last_week, 46)
print(f"Computing incidence for season: {my_season}")

my_yearweek = last_week
#############

# influenza seasons go from Nov 1st to May 1st
min_year = str(YEAR_MIN)
max_year = str(YEAR_MAX)
previous_season = str(int(min_year)-1)+'-'+str(int(max_year)-1)
first_day = min_year+'-11-01' # from Nov 1stM
last_day = max_year+'-05-01' # to May 1st
print(f"Min year: {min_year}, Max year: {max_year}, Prev season: {previous_season}")
print(f"First day: {first_day}, Last day: {last_day}")


In [ ]:
# CALENDAR
delta = datetime.timedelta(days=1)
curr, end = datetime.date(YEAR_MIN, 1, 1), datetime.date(YEAR_MAX, 1, 1)
date_week = dict()
llyearweek, week_to_consider = [], []
while curr <= end:
    yearweek = fix_yearweek( str(curr.isocalendar()[0])+'-'+str(curr.isocalendar()[1]) )
    date = curr.isoformat()
    date_week[date] = yearweek
    curr+=delta

#associate each date to a season
date_season = dict()
season_week = defaultdict(set)
week_season = {}
for yr_min in range(YEAR_MIN, YEAR_MAX, 1):
    yr_max = yr_min+1
    for date in date_week.keys():
        from_ = datetime.date(yr_min, 11, 1).isoformat()
        to_ = datetime.date(yr_max, 5, 1).isoformat()
        if str(yr_min)+'-11-01'<= date <= str(yr_max)+'-05-01' :
            season = str(yr_min)+'-'+str(yr_max)
            date_season[date] = season
            date_f = datetime.datetime.strptime(str(date),'%Y-%m-%d')
            weeknr=str(date_f.isocalendar().year)+'-'+str(date_f.isocalendar().week).zfill(2)
            season_week[season].add(weeknr)
            week_season[weeknr]=season

#############

seasons = set(season_week.keys())
seasons

In [ ]:
#v2
intake_file = path+"/intake_responses_v2.csv"
weekly_file = path+"/weekly_responses_v2.csv"
#v1
intake_file_prev = path_prev+"/intake_responses_v1.csv"
weekly_file_prev = path_prev+"/weekly_responses_v1.csv"
#v0
intake_file_prev2 = path_prev2+"/intake_responses_v0.csv"
weekly_file_prev2 = path_prev2+"/weekly_responses_v0.csv"
            
#############

############
## INTAKE ##
############

#intake v2
intake_complete_now = pd.read_csv(intake_file, sep=',', dtype=str)
#intake v1
intake_complete_prev = pd.read_csv(intake_file_prev, sep=',', dtype=str)
#intake v0
intake_complete_prev2 = pd.read_csv(intake_file_prev2, sep=',', dtype=str)

#the old platform intake survey uses a different format of timestamp, homogenize it to the new ones
intake_complete_prev2['timestamp']=intake_complete_prev2['timestamp'].apply(lambda x: get_epoch(x))
#this creates the column "timestamp" using the pre-defined function get_epoch (top of the notebook)
#apply + lambda function work row-wise on the dataframe

#merge all the versions intakes together
intake_complete=pd.concat([intake_complete_now,intake_complete_prev,intake_complete_prev2])
#transform the timestamp to dates
intake_complete.timestamp = intake_complete.timestamp.apply(lambda d: pd.to_datetime(int(d),unit='s'))
#impose a specific date format
intake_complete.timestamp = pd.to_datetime(intake_complete.timestamp, utc=True).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
#sort by date
intake_complete = intake_complete.sort_values("timestamp", ascending=True)

#country check to remove inconsistencies
if 'country' in intake_complete.columns:
    intake_complete = intake_complete[intake_complete.country == country_code]

    
# some cleaning:
# remove surveys with no global_id , if any
intake_complete = intake_complete[pd.isnull(intake_complete.global_id)==False]

# rename cols
intake_complete.rename(columns={'timestamp':'intake_timestamp', 'Q3':'postal_code'}, inplace=True)
#remove eventual double submissions of the same intake survey from the same user (intake timestamp includes seconds)
intake_complete.drop_duplicates('intake_timestamp', keep='last', inplace=True)

# add date, province and vaccine
# insert is an analogous method of column creation without calling the = operator with apply + lambda function
# this creates the column intake_submission on the 4th column following the rule on the right 
intake_complete.insert(4, "intake_submission", intake_complete.intake_timestamp.str.split().str[0])

#this questions in the "intake survey" up to v1 included, from v2 on it is in the "vaccination" survey
intake_complete['vaccine'] = intake_complete.apply(lambda row: True if row['Q10']==0 else ( False if row['Q10']==1 else 'NN'), axis=1)
intake_complete['vaccine'] = intake_complete['Q10']

# add occupation and education degree
intake_complete[['Q4','Q4d_0','Q4d_1','Q4d_2','Q4d_3','Q4d_4','Q4d_5']] = intake_complete[['Q4','Q4d_0','Q4d_1','Q4d_2','Q4d_3','Q4d_4','Q4d_5']].applymap(lambda x: translate(x))
intake_complete['occupation'] = intake_complete['Q4'].apply(lambda x: occupation(x))
intake_complete['education'] = intake_complete[['Q4d_0','Q4d_1','Q4d_2','Q4d_3','Q4d_4','Q4d_5']].apply(lambda x: unite(x),axis=1)
intake_complete['education'] = intake_complete['education'].apply(lambda x: schooling(x))

# add medical conditions
intake_complete[['Q11_0','Q11_1','Q11_2','Q11_3','Q11_4','Q11_5','Q11_6','Q11_7', 'Q14_1','Q14_2','Q14_3','Q14_4','Q14_5']] = intake_complete[['Q11_0','Q11_1','Q11_2','Q11_3','Q11_4','Q11_5','Q11_6','Q11_7','Q14_1','Q14_2','Q14_3','Q14_4','Q14_5']].applymap(lambda x: translate(x))
intake_complete[['Q12', 'Q13']] = intake_complete[['Q12', 'Q13']].astype(float)
intake_complete[['Q1']] = intake_complete[['Q1']].astype(float)
intake_complete[['Q2']] = intake_complete[['Q2']].astype(str)

# conditions
intake_complete['meds'] = intake_complete.apply(lambda row: True if any([row['Q11_1']==True,row['Q11_2']==True,row['Q11_3']==True,row['Q11_4']==True,row['Q11_5']==True,row['Q11_6']==True]) else False, axis=1)
intake_complete['pregnancy'] = intake_complete.apply(lambda row: True if row['Q12']==0  else False, axis=1)
intake_complete['smoke'] = intake_complete.apply(lambda row: True if any([row['Q13']==1,row['Q13']==2,row['Q13']==3,row['Q13']==5]) else False, axis=1)
intake_complete['allergy'] = intake_complete.apply(lambda row: True if any([row['Q14_1']==True,row['Q14_2']==True,row['Q14_3']==True,row['Q14_4']==True]) else False, axis=1)
intake_complete['gender'] = intake_complete.apply(lambda row: 'M' if row['Q1']==0 else 'F' if row['Q1']==1 else False, axis=1)
intake_complete['age'] = intake_complete.apply(lambda x: get_age(x),axis=1)
intake_complete['age_class'] = intake_complete[['age']].applymap(lambda x: get_age_class(x))
intake_complete = intake_complete[intake_complete.age_class!='nan']

intake = intake_complete[["global_id","intake_timestamp","intake_submission","meds","pregnancy","smoke","allergy","gender","age_class","occupation","education"]]


In [ ]:
#visualize resulting dataframe first five rows
print(intake.head())

In [ ]:
############
## WEEKLY ##
############
#v2 weekly survey 
weekly_complete_now = pd.read_csv(weekly_file, sep=',', dtype=str)         #dates in timestamp
#v1 weekly survey 
weekly_complete_prev = pd.read_csv(weekly_file_prev, sep=',', dtype=str)   #dates in timestamp
#v0 weekly survey 
weekly_complete_prev2 = pd.read_csv(weekly_file_prev2, sep=',', dtype=str) #dates in epoch time
#the old platform weekly results consider a different format of timestamp, homogenize it to the new ones
weekly_complete_prev2['timestamp']=weekly_complete_prev2['timestamp'].apply(lambda x: get_epoch(x)) #conver to dates

def transf_date(x):
    if pd.notnull(x):
        newx = pd.to_datetime(int(x),unit='s')
        return newx
    else:
        return ''

#transform onset, fever and end of symptoms epoch times to datetime, adding NaN where unavailable  
weekly_complete_prev['Q3_0_open'] = weekly_complete_prev['Q3_0_open'].apply(lambda d: transf_date(d))
weekly_complete_prev['Q4_0_open'] = weekly_complete_prev['Q4_0_open'].apply(lambda d: transf_date(d))
weekly_complete_prev['Q6_1_open'] = weekly_complete_prev['Q6_1_open'].apply(lambda d: transf_date(d))
weekly_complete_now['Q3_0_open'] = weekly_complete_now['Q3_0_open'].apply(lambda d: transf_date(d))
weekly_complete_now['Q4_0_open'] = weekly_complete_now['Q4_0_open'].apply(lambda d: transf_date(d))
weekly_complete_now['Q6_1_open'] = weekly_complete_now['Q6_1_open'].apply(lambda d: transf_date(d))

#merge all versions
weekly_complete=pd.concat([weekly_complete_now,weekly_complete_prev,weekly_complete_prev2])
#submission epoch time to datetimes
weekly_complete.timestamp = weekly_complete.timestamp.apply(lambda d: pd.to_datetime(int(d),unit='s', errors = 'coerce'))
#format submission datetime dates to desired format
weekly_complete.timestamp = pd.to_datetime(weekly_complete.timestamp, utc=True).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
#sort dates
weekly_complete = weekly_complete.sort_values("timestamp", ascending=True)
#remove those not complying with format
weekly_complete = weekly_complete[weekly_complete.timestamp <= dates.strftime('%Y-%m-%d %H:%M:%S')]

#apply same format for other answers with dates
weekly_complete['Q3_0_open'] = pd.to_datetime(weekly_complete['Q3_0_open'], utc=True, errors='coerce').apply(lambda d: d.strftime('%Y-%m-%d'))
weekly_complete['Q4_0_open'] = pd.to_datetime(weekly_complete['Q4_0_open'], utc=True, errors='coerce').apply(lambda d: d.strftime('%Y-%m-%d'))
weekly_complete['Q6_1_open'] = pd.to_datetime(weekly_complete['Q6_1_open'], utc=True, errors='coerce').apply(lambda d: d.strftime('%Y-%m-%d'))

#homogenize true and false in symptoms answers
weekly_complete[['Q1_0','Q1_1','Q1_2','Q1_11','Q1_8','Q1_9','Q1_5','Q1_6','Q1_7','Q1_15','Q1_16','Q1_17','Q1_20']]= weekly_complete[['Q1_0','Q1_1','Q1_2','Q1_11','Q1_8','Q1_9','Q1_5','Q1_6','Q1_7','Q1_15','Q1_16','Q1_17','Q1_20']].applymap(lambda x: translate(x))
weekly_complete[['Q1_21', 'Q1_23']] = weekly_complete[['Q1_21', 'Q1_23']].applymap(lambda x: translate(x))
#same for questions on healthcare seeking 
weekly_complete[['Q7_1','Q7_2','Q7_3','Q7_4','Q8_1','Q8_2','Q8_3','Q8_4','Q8_5']]= weekly_complete[['Q7_1','Q7_2','Q7_3','Q7_4','Q8_1','Q8_2','Q8_3','Q8_4','Q8_5']].applymap(lambda x: translate(x))


weekly_complete[['Q5', 'Q6b']] = weekly_complete[['Q5', 'Q6b']].astype(float)
weekly_complete[['Q11']] = weekly_complete[['Q11']].astype(float)

#homogeneize covid testing questions
weekly_complete[['Qcov16_1', 'Qcov16_2', 'Qcov16_5']] = weekly_complete[['Qcov16_1', 'Qcov16_2', 'Qcov16_5']].applymap(lambda x: translate(x))

# some cleaning:
# remove surveys with no global_id, if any, and consider only surveys which have a corresponding intake survey
weekly_complete = weekly_complete[pd.isnull(weekly_complete.global_id)==False]
weekly_complete = weekly_complete[weekly_complete.global_id.isin(intake.global_id.unique())]

# add cols
weekly_complete.rename(columns={'timestamp': 'weekly_timestamp'}, inplace=True)
weekly_complete.insert(3, "submission_date", weekly_complete.weekly_timestamp.str.split().str[0])
weekly_complete.insert(4, "submission_week", weekly_complete.submission_date.map(date_week))
weekly_complete.insert(5, "season", weekly_complete.submission_date.map(date_season))

# consider only data for the seasons given in input 
weekly = weekly_complete[ weekly_complete.season.isin(seasons) ]
        
# remove duplicates within the same week, keeping the last one (user may have updated their symptoms)
weekly = weekly_complete.drop_duplicates(['global_id','submission_week'], keep='last', inplace=False)


In [ ]:
#####################
## WEEKLY + INTAKE ##
#####################
# Merge weekly and intake according to the most recent intake per each weekly_survey submitted
frames = []
for item, group in weekly.groupby(["global_id","weekly_timestamp","submission_date"]):
    global_id, weekly_timestamp, submission_date = item
    #for each user consider the most recent intake submission with respect to each weekly survey submission (medical conditions, age, etc may change)
    intake_timestamp = intake[(intake.global_id==global_id) & (intake.intake_timestamp<=weekly_timestamp)].intake_timestamp.max()
    frames.append({'global_id': global_id, 'submission_date':submission_date, 'intake_timestamp':intake_timestamp})
data = weekly.merge(pd.DataFrame(frames), on=["global_id", "submission_date"], how="left")
data = data.merge(intake, on=["global_id", "intake_timestamp"], how="left")
assert(data.shape[0]==weekly.shape[0])
data.head()

### get age-classes populations from census
http://dati.istat.it/Index.aspx?QueryId=42869#

In [ ]:

    
age_pop=pd.read_csv('../schools_int_mobility/DCIS_POPRES1_31082023121420138.csv')
age_pop=age_pop[age_pop.ETA1!='TOTAL']
age_pop=age_pop[age_pop.Territorio=='Italia']
age_pop=age_pop[age_pop.Sesso=='totale']

age_pop['age_class'] = age_pop.ETA1.apply(lambda x: get_iss_age_classes(x))
ita_pop=58850717
age_class_pop=age_pop[['age_class','Value']].groupby('age_class')['Value'].sum()/ita_pop
print( sum(age_class_pop) == 1 )


In [ ]:
# ACTIVE USERS inclusion criteria:
# - AT LEAST 2 symptoms surveys
# - we count users as active from: +/- 2 weeks around the week of symptoms reporting

# keep only surveys of participants who submitted >= 2 symptoms surveys
data = data.groupby('global_id').filter(lambda x: len(x)>1)
if data.empty:
    sys.exit('### no active users ###\n')

# get num. of active users per week
weekly_active_user = {}
weekly_active_user_age_list = defaultdict(list)
for global_id, group in data.groupby('global_id'):
    #this counts the +/- 2 weeks around the submitted sympotms date
    activity_weeks = get_week_of_activity(global_id, group.submission_week)
    age=group.age_class.unique()[0]
    #consider users with valid dates
    if str(age)!='nan': 
        for wk in activity_weeks:
            weekly_active_user.setdefault(wk, 0)
            weekly_active_user[wk] += 1 #sum weekly active users as those posting in
            weekly_active_user_age_list[wk].append(age) #count how many users we have per age class

In [ ]:
#define the age correction factors, observed vs expected age class composition from census
weekly_active_user_age_correction = defaultdict(dict)

for wk in weekly_active_user_age_list:
    age_counter=Counter(weekly_active_user_age_list[wk])
    for age in age_counter:
        weekly_active_user_age_correction[wk][age] = (age_counter[age]/weekly_active_user[wk]) / age_class_pop[age]
        

In [ ]:
# ILI cases inclusion criteria:
# - get ILI among active users
# - EXCLUSION OF FIRST ILI EPISODE if reported in the 1st symptoms survey of the season
# - onset of ILI is symptoms_onset if reported and not older than 15 days
# - keep only 1st episode if the participants reported prolonged ILI status with the same onset_week

# define first_survey
data['first_survey'] = data.groupby(['global_id'])['submission_week'].transform(np.min)

In [ ]:

data_ILI = data.copy(deep=True)

## !!! python warning
## data.copy(deep=True) allows to create a deep copy of the dataframe 
## setting new_df = data (without the deep copy approach) creates a new dataframe "new_df":
## when you modify new_df, changes will apply on the old dataframe "data" as well
## setting new_df = data.copy(deep==True) prevents this

### only ILI

In [ ]:
data_ILI = data_ILI[ data_ILI.season.isin(seasons) ] #get only weeks in seasons

ILI_weeks=set(data_ILI.submission_week)
#influenza season weeks
submission_weeks=[x for x in list(week_season.keys()) if x<=last_week] 
#full year weeks
submission_weeks_complete=set(data.submission_week) 

#create column ILI
data_ILI['ILI'] = data_ILI.apply(lambda row: get_ILI_ECDC(row), axis=1)

data_ILI['phlegm'] = data_ILI.apply(lambda row: get_phlegm(row), axis=1)
data_ILI['fever'] = data_ILI.apply(lambda row: get_fever(row), axis=1)


#remove first symptomatic submission ever
data_ILI.loc[(data_ILI['ILI']==True) & (data_ILI['submission_week']==data_ILI['first_survey']), 'ILI']= False
#define onset week (if submitted, else use fever data, else nan)
data_ILI['onset_week'] = data_ILI.apply(lambda row: get_onset_date(row.submission_date, row['Q3_0_open'], row['Q6_1_open']) if row['ILI']==True else np.nan, axis=1).map(date_week) # only if ILI

# check for multiple episodes in the same onset week
if data_ILI[data_ILI['ILI']==True].shape[0]>0:
    tmp = data_ILI[data_ILI.ILI==True].groupby(['global_id','onset_week'])['submission_week'].unique().reset_index(name='submission_weeks')
    tmp['count'] = tmp['submission_weeks'].apply(lambda x: len(x))
    # keep only the 1st episode if symptoms continue in the next weeks:
    for global_id, onset_week, these_submission_weeks in tmp[tmp['count']>1][['global_id','onset_week','submission_weeks']].values:
        first_episode = np.min(these_submission_weeks)
        for wk in these_submission_weeks:
            if wk!=first_episode:
                data_ILI.loc[((data_ILI.global_id==global_id) & (data_ILI.submission_week==wk) & (data_ILI.ILI==True) & (data_ILI.onset_week==onset_week)), 'ILI'] = False # then turn ILI to False
                #debug check, if none appear, there may NaNs in the onset and submission week columns
                print('removed 1')



In [ ]:
#compute ILI incidence

active, ILI = 0, 0
incidence = {}
act_threshold = 100 #(optional) minimum number of weekly active users to compute incidence

for week in sorted(submission_weeks):
    active, ILI = 0, 0
    if week in weekly_ILI:
        active = weekly_active_user[week]
        ILI = weekly_ILI[week]
    else: ILI = 0
    if active>act_threshold and ILI>0:
        incidence[week] = round( ILI*1.0/active*rescaling, 2 )
    else: incidence[week] = 0

In [ ]:
#define health seeking column
#creates column medical_service if any of the conditions on the right if fulfilled, else False
data_ILI["medical_service"] = data_ILI.apply(lambda row: True if row['Q7_1']==True or row['Q7_2']==True or row['Q7_3']==True or row['Q7_4']==True or row['Q8_1']==True or row['Q8_2']==True or row['Q8_3']==True or row['Q8_4']==True or row['Q8_5']==True else False, axis=1)
#apply + lambda function work row-wise, axis=1 tells the lambda function that Q7_1 etc are columns names

#other examples
data_ILI["medical_service_inperson"] = data_ILI.apply(lambda row: True if row['Q7_1']==True or row['Q7_2']==True or row['Q7_3']==True or row['Q7_4']==True else False, axis=1)
data_ILI["medical_service_remote"] = data_ILI.apply(lambda row: True if row['Q8_1']==True or row['Q8_2']==True or row['Q8_3']==True or row['Q8_4']==True or row['Q8_5']==True else False, axis=1)
data_ILI["flu_test"] = data_ILI.apply(lambda row: True if row['Qcov19']=='1' else False, axis=1)
data_ILI["cov_test"] = data_ILI.apply(lambda row: True if row['Qcov16h']=='1' or row['Qcov16_1']==True or row['Qcov16_2']==True or row['Qcov16_5']==True else False, axis=1)


In [ ]:
#count seasonal healthseeking behavior
#the row below reads as: 
#get all data_ILI entries that fulfill the condition  ILI==True and then group them by season and count how many they are
season_ILI = data_ILI[(data_ILI.ILI==True)].groupby('season').size()
#result is the number of ILI episodes in each season
#note that we kept one ILI episode per user per onset week(the 1st one)
#this count should change if we want to keep more ILI reports after the 1st week of submission

#same here but with two conditions
season_ILIfever = data_ILI[(data_ILI.ILI==True)&(data_ILI.fever==True)].groupby('season').size()
season_ILInofever = data_ILI[(data_ILI.ILI==True)&(data_ILI.fever==False)].groupby('season').size()
season_ILIfever_phlegm = data_ILI[(data_ILI.ILI==True)&(data_ILI.fever==True)&(data_ILI.phlegm==True)].groupby('season').size()
    
#this is pretty much what you need to know on how python and pandas work

In [ ]:
weekly_ILI = data_ILI[(data_ILI.ILI==True)].groupby('onset_week').size()